In [1]:
!pip install ffmpeg

  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=368b2b4fbed232a916c5163798d07aa78bd8c52b5f51e29cb86d32bdd601ab3e
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# We'll need numpy for some mathematical operations
import numpy as np

# matplotlib for displaying the output
import matplotlib.pyplot as plt
%matplotlib inline

# and IPython.display for audio output
import IPython.display

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Trying to find the difference between the melspectrogram and mfcc command in the librosa library.
It turns out that the MFCC creates pixels which represent the **rate of change of frequency** for that time interval and frequency band, while the Mel spectrogram is just a Fourier transform spectrogram with a set of filters to try and emphasize the frequencies that humans hear best.

In [ ]:
#Using the melspectrogram as used by some chap on kaggle
path = "/kaggle/input/birdsong-recognition/train_audio/pinjay/XC380627.mp3"
#'../input/birdsong-recognition/train_audio/lotduc/XC195038.mp3' #The file that threw up errors during the 5Hrs test run
audio, sample_rate = librosa.load(path, res_type='kaiser_fast')
audio = audio[0:80000] #triming the audio down to length in number of samples
mfccs = librosa.feature.melspectrogram(y=audio, sr=sample_rate,  n_mels=5)  #Reducing the resolution of the sepctrogram n_mfcc=40 ---> n_mfcc=20
# Convert to log scale (dB). We'll use the peak power (max) as reference.
log_S = librosa.power_to_db(mfccs, ref=np.max)
print(np.shape(log_S))
# Make a new figure
plt.figure(figsize=(12,4))

# Display the spectrogram on a mel scale
# sample rate and hop length parameters are used to render the time axis
librosa.display.specshow(log_S, sr=sample_rate, x_axis='time', y_axis='mel')

# Put a descriptive title on the plot
plt.title('mel power spectrogram using command librosa.feature.melspectrogram')

# draw a color bar
plt.colorbar(format='%+02.0f dB')

# Make the figure layout compact
plt.tight_layout()

In [ ]:
#Using MFCC as used in mikes code
path = "/kaggle/input/birdsong-recognition/train_audio/pinjay/XC380627.mp3"#'../input/birdsong-recognition/train_audio/lotduc/XC195038.mp3' #The file that threw up errors
audio, sample_rate = librosa.load(path, res_type='kaiser_fast')
print(len(audio))
print(sample_rate)
print("The length of the audio segment we are testing")
print(10000/sample_rate)
mfccs = librosa.feature.mfcc(y=audio[0:66150], sr=sample_rate,  n_mfcc=40)  #Reducing the resolution of the sepctrogram n_mfcc=40 ---> n_mfcc=20
#mfccsscaled = np.mean(mfccs.T,axis=0)
print(np.shape(mfccs))

# Make a new figure
plt.figure(figsize=(12,4))

# Display the spectrogram on a mel scale
# sample rate and hop length parameters are used to render the time axis
librosa.display.specshow(mfccs, sr=sample_rate, x_axis='time', y_axis='mel')

# Put a descriptive title on the plot
plt.title('spectrogram using command librosa.feature.mfcc as in Mikes code')

# draw a color bar
plt.colorbar(format='%+02.0f dB')

# Make the figure layout compact
plt.tight_layout()

# Loading the CSV and making numerical labels

In [3]:
#Load the data into a Pandas data frame so that we can attach a classification label to each spectrogram for training
train = pd.read_csv("../input/birdsong-recognition/train.csv")
#train.info()
ebird_code = train['ebird_code']
#remove duplicate ebird codes
ebird_code = list(dict.fromkeys(ebird_code)) 
#put ebird codes into a dict mapping each code to a number
ebird_dict = { ebird_code[i] : i for i in range(len(ebird_code)) }
#print(ebird_dict)
#Using ebird_code instead of species becuase it's a bit shorter. 
#But these will have to be converted to numbers if we want to use tensorflow

# Extracting features from the Audio using a spectrogram

In [49]:
import warnings
warnings.filterwarnings("ignore", message="PySoundFile failed") #Actually don't think this is needed now we imported ffmpeg but idk yet

import numpy as np
max_pad_len = 135

%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display


def extract_features2(file_name): #returns a list of mfccs
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        length = librosa.get_duration(audio)
        mfccs_list = []
        #get middle 3 seconds of each audio
        #the audio will be of the form: x seconds + 3 middle seconds + x seconds, so x (the start index of the middle 3 seconds) is given by (length-3)/2
        #start = (length-3.0)/2   
        #end = start + 3
        #one second has 22050 samples so multiply start and end time in seconds by that to get it in samples and round to nearest integer
        #audio = audio[round(22050*start):round(22050*end)]
        for start in range(round(length)-2): #windowing: take 3s segments with 1s offsets
            end = start + 3
            cur_audio = audio[round(22050*start):round(22050*end)]
            mfccs = librosa.feature.mfcc(y=cur_audio, sr=sample_rate, n_mfcc=40) #[0:66150]
            pad_width = max_pad_len - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
            mfccs_list.append(mfccs)
        #mfccs = librosa.feature.mfcc(y=audio[0:66150], sr=sample_rate, n_mfcc=40)
        #pad_width = max_pad_len - mfccs.shape[1]
        #mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        #print(mfccs.shape)
        #plt.figure(figsize=(14, 5))
        #librosa.display.waveplot(audio, sr=sample_rate)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs_list

path = "/kaggle/input/birdsong-recognition/train_audio/pinjay/XC380627.mp3"
extract_features2(path)



[array([[-498.84402   , -412.64713   , -383.4195    , ...,    0.        ,
            0.        ,    0.        ],
        [ 112.557884  ,  122.34829   ,  134.43396   , ...,    0.        ,
            0.        ,    0.        ],
        [   6.9547663 ,    8.98547   ,   12.925907  , ...,    0.        ,
            0.        ,    0.        ],
        ...,
        [   7.307838  ,   10.070247  ,    8.075116  , ...,    0.        ,
            0.        ,    0.        ],
        [   5.441354  ,    4.305888  ,    1.1831048 , ...,    0.        ,
            0.        ,    0.        ],
        [  -6.5151434 ,   -0.51517594,   -1.2354028 , ...,    0.        ,
            0.        ,    0.        ]], dtype=float32),
 array([[-3.68922180e+02, -3.57789825e+02, -3.68061249e+02, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.27290871e+02,  1.23352676e+02,  1.26266365e+02, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-7.68322372e+00, -9.05374

In [50]:
features = []
for dirname, _, filenames in os.walk('/kaggle/input/birdsong-recognition/train_audio/buwwar'):
    for filename in filenames:
        try:
            '''The 2 lines below go into Lioras for loop that iterates through the directory '''
            print(os.path.join(dirname, filename))
            data_list = extract_features2(os.path.join(dirname, filename))
            row = train.loc[train['filename'] == filename]
            ebird = row['ebird_code'].tolist() #You need this because otherwise it returns the row index too
            class_label = ebird_dict[ebird[0]]
            #class_label = filename #TEMPORARY OK 
            for data in data_list:
                features.append([data, class_label])
            #print(filename)
        except Exception as e:
            print(e)
            
for dirname, _, filenames in os.walk('/kaggle/input/birdsong-recognition/train_audio/semplo'):
    for filename in filenames:
        try:
            '''The 2 lines below go into Lioras for loop that iterates through the directory '''
            print(os.path.join(dirname, filename))
            data_list = extract_features2(os.path.join(dirname, filename))
            row = train.loc[train['filename'] == filename]
            ebird = row['ebird_code'].tolist() #You need this because otherwise it returns the row index too
            class_label = ebird_dict[ebird[0]]
            #class_label = filename #TEMPORARY OK 
            for data in data_list:
                features.append([data, class_label])
            #print(filename)
        except Exception as e:
            print(e)
            
for dirname, _, filenames in os.walk('/kaggle/input/birdsong-recognition/train_audio/ruckin'):
    for filename in filenames:
        try:
            '''The 2 lines below go into Lioras for loop that iterates through the directory '''
            print(os.path.join(dirname, filename))
            data_list = extract_features2(os.path.join(dirname, filename))
            row = train.loc[train['filename'] == filename]
            ebird = row['ebird_code'].tolist() #You need this because otherwise it returns the row index too
            class_label = ebird_dict[ebird[0]]
            #class_label = filename #TEMPORARY OK 
            for data in data_list:
                features.append([data, class_label])
            #print(filename)
        except Exception as e:
            print(e)
print(np.shape(features))  #21375, 2

print(features[0:5])
# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

/kaggle/input/birdsong-recognition/train_audio/buwwar/XC497156.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC318332.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC296164.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC179095.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC137741.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC254299.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC236555.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC78891.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC444716.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC135501.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC254295.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC31124.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC241103.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC31313.mp3
/kaggle/input/birdsong-recognition/train_audio/buwwar/XC385393.mp

/kaggle/input/birdsong-recognition/train_audio/semplo/XC131020.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC109656.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC149423.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC372697.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC149624.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC257555.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC331260.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC341985.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC191777.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC406390.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC492111.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC149625.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC494941.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC149421.mp3
/kaggle/input/birdsong-recognition/train_audio/semplo/XC155923

/kaggle/input/birdsong-recognition/train_audio/ruckin/XC291510.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC138055.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC131643.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC289677.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC182072.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC193049.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC175994.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC191466.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC466900.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC373500.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC369882.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC201424.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC251808.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC184996.mp3
/kaggle/input/birdsong-recognition/train_audio/ruckin/XC185185

Find the dimensions of a spectrogram once the audio has been chopped down. Turns out when chopped to 80000 samples (3.6 seconds) and using n_mfccs = 5, the shape of the spectrogram is (5,157).
With 10000 sample (0.45 seconds), the spectrogram has size (5,20).

In [51]:
print(features[0][0]) #This should be (40,135)
print(np.shape(features[0][0]))
print(extract_features2("/kaggle/input/birdsong-recognition/train_audio/buwwar/XC475302.mp3"))

[[-6.69588135e+02 -7.06461365e+02 -7.49672668e+02 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.71937103e+02 -2.01719940e+02 -2.32784485e+02 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.04630302e+02  1.15159523e+02  1.26668045e+02 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 1.83603740e+00  1.00618458e+00 -1.52757436e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.87633410e-01  1.13081145e+00  3.12737882e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 2.13314605e+00  1.97001112e+00 -7.71971226e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
(40, 135)
[array([[-294.8577   , -291.32312  , -296.5393   , ...,    0.       ,
           0.       ,    0.       ],
       [  94.32274  ,   98.09225  ,   96.408936 , ...,    0.       ,
           0.       ,    0.       ],
       [  -6.1629486,   -9.458319 ,  -10.757315 , ...,    0.       ,
           0.       ,    0.       ],
       ...,
       [  -5.1183147,  

# Liora ur saving code will go under here

In [52]:
featuresdf.to_pickle("/kaggle/working/features_windowing.pkl")
print(featuresdf)  

                                                 feature  class_label
0      [[-669.58813, -706.46136, -749.67267, -756.449...           53
1      [[-603.33813, -651.0848, -810.6111, -816.2782,...           53
2      [[-626.37146, -672.08044, -811.8544, -810.9829...           53
3      [[-618.4137, -662.8052, -787.5116, -784.623, -...           53
4      [[-605.1233, -646.90594, -775.7363, -783.81964...           53
...                                                  ...          ...
12668  [[-389.48938, -386.13083, -385.13953, -378.964...          203
12669  [[-353.81934, -356.7925, -364.82898, -366.9326...          203
12670  [[-333.76367, -319.9668, -330.44907, -339.4731...          203
12671  [[-374.1242, -373.10034, -374.64334, -371.6339...          203
12672  [[-376.0453, -367.81967, -372.85208, -370.8771...          203

[12673 rows x 2 columns]


# Reload the feautures dataframe for the next cells to use

In [5]:
featuresdf = pd.read_pickle("/kaggle/working/features.pkl")
print(featuresdf) 
print(np.shape(featuresdf))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/features.pkl'

# Converting the data into a form that is compatible with Tensorflow

In [53]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
print(np.shape(X))
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)


(12673, 40, 135)


# Generating a CNN

**Convolutional Neural Network (CNN) model architecture**
We will modify our model to be a Convolutional Neural Network (CNN) again using Keras and a Tensorflow backend.

Again we will use a sequential model, starting with a simple model architecture, consisting of four Conv2D convolution layers, with our final output layer being a dense layer.

The convolution layers are designed for feature detection. It works by sliding a filter window over the input and performing a matrix multiplication and storing the result in a feature map. This operation is known as a convolution.

The filter parameter specifies the number of nodes in each layer. Each layer will increase in size from 16, 32, 64 to 128, while the kernel_size parameter specifies the size of the kernel window which in this case is 2 resulting in a 2x2 filter matrix.

The first layer will receive the input shape of (40, 174, 1) where 40 is the number of MFCC's 174 is the number of frames taking padding into account and the 1 signifying that the audio is mono.

The activation function we will be using for our convolutional layers is ReLU which is the same as our previous model. We will use a smaller Dropout value of 20% on our convolutional layers.

Each convolutional layer has an associated pooling layer of MaxPooling2D type with the final convolutional layer having a GlobalAveragePooling2D type. The pooling layer is do reduce the dimensionality of the model (by reducing the parameters and subsquent computation requirements) which serves to shorten the training time and reduce overfitting. The Max Pooling type takes the maximum size for each window and the Global Average Pooling type takes the average which is suitable for feeding into our dense output layer.

Our output layer will have 10 nodes (num_labels) which matches the number of possible classifications. The activation is for our output layer is softmax. Softmax makes the output sum up to 1 so the output can be interpreted as probabilities. The model will then make its prediction based on which option has the highest probability.

In [54]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 
 
num_rows = 40
'''This part will depend on the feature extraction'''
num_columns = max_pad_len 
'''This part will depend on the length of the audio segment (adjusted in the Feature_extraction function)'''
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)
print(np.shape(x_train))
print(np.shape(x_test))
print(np.shape(y_test))



num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2)) #Has a problem with this line (THE INPUT IMAGE WAS TOO SMALL)
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2)) ###
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

(10138, 40, 135, 1)
(2535, 40, 135, 1)
(2535, 3)


In [55]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()
print(y_test)
# Calculate pre-training accuracy 

score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 39, 134, 16)       80        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 19, 67, 16)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 19, 67, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 18, 66, 32)        2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 33, 32)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 9, 33, 32)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 32, 64)        

In [56]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
from numpy.random import seed
from tensorflow.random import set_seed

#set seeds in hopes that it'll allow for reproducible results
seed(1)
set_seed(2)

#num_epochs = 12
#num_batch_size = 128

#num_epochs = 72
#num_batch_size = 256

num_epochs = 150
num_batch_size = 512

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/150
20/20 [==============================] - ETA: 0s - loss: 4.5678 - accuracy: 0.4411
Epoch 00001: val_loss improved from inf to 0.96638, saving model to saved_models/weights.best.basic_cnn.hdf5
20/20 [==============================] - 19s 946ms/step - loss: 4.5678 - accuracy: 0.4411 - val_loss: 0.9664 - val_accuracy: 0.5002
Epoch 2/150
20/20 [==============================] - ETA: 0s - loss: 1.5973 - accuracy: 0.5042
Epoch 00002: val_loss improved from 0.96638 to 0.90158, saving model to saved_models/weights.best.basic_cnn.hdf5
20/20 [==============================] - 19s 941ms/step - loss: 1.5973 - accuracy: 0.5042 - val_loss: 0.9016 - val_accuracy: 0.5538
Epoch 3/150
20/20 [==============================] - ETA: 0s - loss: 1.1769 - accuracy: 0.5378
Epoch 00003: val_loss improved from 0.90158 to 0.81808, saving model to saved_models/weights.best.basic_cnn.hdf5
20/20 [==============================] - 18s 924ms/step - loss: 1.1769 - accuracy: 0.5378 - val_loss: 0.8181 - val_a

Epoch 26/150
20/20 [==============================] - ETA: 0s - loss: 0.3352 - accuracy: 0.8737
Epoch 00026: val_loss improved from 0.33653 to 0.33040, saving model to saved_models/weights.best.basic_cnn.hdf5
20/20 [==============================] - 18s 890ms/step - loss: 0.3352 - accuracy: 0.8737 - val_loss: 0.3304 - val_accuracy: 0.8982
Epoch 27/150
20/20 [==============================] - ETA: 0s - loss: 0.3109 - accuracy: 0.8791
Epoch 00027: val_loss improved from 0.33040 to 0.30903, saving model to saved_models/weights.best.basic_cnn.hdf5
20/20 [==============================] - 19s 925ms/step - loss: 0.3109 - accuracy: 0.8791 - val_loss: 0.3090 - val_accuracy: 0.9034
Epoch 28/150
20/20 [==============================] - ETA: 0s - loss: 0.3046 - accuracy: 0.8842
Epoch 00028: val_loss improved from 0.30903 to 0.30198, saving model to saved_models/weights.best.basic_cnn.hdf5
20/20 [==============================] - 18s 894ms/step - loss: 0.3046 - accuracy: 0.8842 - val_loss: 0.3020 

Epoch 52/150
20/20 [==============================] - ETA: 0s - loss: 0.1425 - accuracy: 0.9487
Epoch 00052: val_loss improved from 0.12954 to 0.12599, saving model to saved_models/weights.best.basic_cnn.hdf5
20/20 [==============================] - 19s 949ms/step - loss: 0.1425 - accuracy: 0.9487 - val_loss: 0.1260 - val_accuracy: 0.9602
Epoch 53/150
20/20 [==============================] - ETA: 0s - loss: 0.1516 - accuracy: 0.9417
Epoch 00053: val_loss did not improve from 0.12599
20/20 [==============================] - 18s 910ms/step - loss: 0.1516 - accuracy: 0.9417 - val_loss: 0.1429 - val_accuracy: 0.9578
Epoch 54/150
20/20 [==============================] - ETA: 0s - loss: 0.1487 - accuracy: 0.9465
Epoch 00054: val_loss did not improve from 0.12599
20/20 [==============================] - 18s 925ms/step - loss: 0.1487 - accuracy: 0.9465 - val_loss: 0.1384 - val_accuracy: 0.9554
Epoch 55/150
20/20 [==============================] - ETA: 0s - loss: 0.1375 - accuracy: 0.9488
Epoch

Epoch 79/150
20/20 [==============================] - ETA: 0s - loss: 0.0824 - accuracy: 0.9709
Epoch 00079: val_loss did not improve from 0.07238
20/20 [==============================] - 19s 971ms/step - loss: 0.0824 - accuracy: 0.9709 - val_loss: 0.0737 - val_accuracy: 0.9744
Epoch 80/150
20/20 [==============================] - ETA: 0s - loss: 0.0835 - accuracy: 0.9701
Epoch 00080: val_loss did not improve from 0.07238
20/20 [==============================] - 19s 955ms/step - loss: 0.0835 - accuracy: 0.9701 - val_loss: 0.0758 - val_accuracy: 0.9755
Epoch 81/150
20/20 [==============================] - ETA: 0s - loss: 0.0828 - accuracy: 0.9696
Epoch 00081: val_loss did not improve from 0.07238
20/20 [==============================] - 19s 949ms/step - loss: 0.0828 - accuracy: 0.9696 - val_loss: 0.0842 - val_accuracy: 0.9736
Epoch 82/150
20/20 [==============================] - ETA: 0s - loss: 0.0829 - accuracy: 0.9737
Epoch 00082: val_loss did not improve from 0.07238
20/20 [=========

20/20 [==============================] - ETA: 0s - loss: 0.0516 - accuracy: 0.9824
Epoch 00107: val_loss did not improve from 0.04940
20/20 [==============================] - 19s 957ms/step - loss: 0.0516 - accuracy: 0.9824 - val_loss: 0.0527 - val_accuracy: 0.9842
Epoch 108/150
20/20 [==============================] - ETA: 0s - loss: 0.0512 - accuracy: 0.9827
Epoch 00108: val_loss improved from 0.04940 to 0.04877, saving model to saved_models/weights.best.basic_cnn.hdf5
20/20 [==============================] - 18s 923ms/step - loss: 0.0512 - accuracy: 0.9827 - val_loss: 0.0488 - val_accuracy: 0.9842
Epoch 109/150
20/20 [==============================] - ETA: 0s - loss: 0.0610 - accuracy: 0.9784
Epoch 00109: val_loss improved from 0.04877 to 0.03913, saving model to saved_models/weights.best.basic_cnn.hdf5
20/20 [==============================] - 18s 913ms/step - loss: 0.0610 - accuracy: 0.9784 - val_loss: 0.0391 - val_accuracy: 0.9886
Epoch 110/150
20/20 [=============================

Epoch 136/150
20/20 [==============================] - ETA: 0s - loss: 0.0366 - accuracy: 0.9870
Epoch 00136: val_loss did not improve from 0.03288
20/20 [==============================] - 18s 898ms/step - loss: 0.0366 - accuracy: 0.9870 - val_loss: 0.0344 - val_accuracy: 0.9882
Epoch 137/150
20/20 [==============================] - ETA: 0s - loss: 0.0417 - accuracy: 0.9865
Epoch 00137: val_loss improved from 0.03288 to 0.03013, saving model to saved_models/weights.best.basic_cnn.hdf5
20/20 [==============================] - 18s 879ms/step - loss: 0.0417 - accuracy: 0.9865 - val_loss: 0.0301 - val_accuracy: 0.9909
Epoch 138/150
20/20 [==============================] - ETA: 0s - loss: 0.0347 - accuracy: 0.9886
Epoch 00138: val_loss did not improve from 0.03013
20/20 [==============================] - 18s 894ms/step - loss: 0.0347 - accuracy: 0.9886 - val_loss: 0.0333 - val_accuracy: 0.9921
Epoch 139/150
20/20 [==============================] - ETA: 0s - loss: 0.0349 - accuracy: 0.9869
E

# Model Evaluation

In [57]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

#Run 1
#Only used audio segments 0.45seconds long
#num_epochs = 12
#num_batch_size = 128
##Training Accuracy:  0.4951
#Testing Accuracy:  0.37735


#Run 2
#with 3 second audio segments trained on 3 different birds
#num_epochs = 12
#num_batch_size = 128
##Training Accuracy:  0.4134615361690521
#Testing Accuracy:  0.3962264060974121

#Run 3
#Increased the training:
#num_epochs = 72
#num_batch_size = 256
#Training Accuracy:  0.6105769276618958
#Testing Accuracy:  0.5849056839942932

#Run 4
#Increased the training:
#num_epochs = 150
#num_batch_size = 256
#Training Accuracy:  0.7403846383094788
#Testing Accuracy:  0.6603773832321167

#Run 5
#Increased the training:
#num_epochs = 150
#num_batch_size = 512
#Training Accuracy:  0.9855769276618958
#Testing Accuracy:  0.7924528121948242

#Run 6
#Middle 3s instead of first 3s:
#num_epochs = 150
#num_batch_size = 512
#Training Accuracy:  0.9759615659713745
#Testing Accuracy:  0.8113207817077637

#Run 7
#Windows of 3s with offset 1s:
#num_epochs = 150
#num_batch_size = 512
#Training Accuracy:  0.9952653646469116
#Testing Accuracy:  0.9905325174331665

#Accuracies vary significantly from run to run due to random elements - setting the seed to a particular value doesn't seem to fix it.

Training Accuracy:  0.9952653646469116
Testing Accuracy:  0.9905325174331665
